<a href="https://colab.research.google.com/github/arpitsharma2814/Project-AI/blob/main/Bayesian_Network_for_Medical_Diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pgmpy pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Step 1: Defining the Bayesian Network structure
# A simple model where Disease -> Symptom1, Symptom2
model = BayesianNetwork([
    ('Disease', 'Fever'),
    ('Disease', 'Cough'),
    ('Disease', 'Fatigue')
])

# Step 2: Defining the Conditional Probability Distributions (CPDs)
# Probability of having a disease (e.g., flu)
cpd_disease = TabularCPD(variable='Disease', variable_card=2, values=[[0.6], [0.4]])  # No Disease, Disease

# Fever depends on Disease (True/False for Disease -> Fever)
cpd_fever = TabularCPD(variable='Fever', variable_card=2,
                       values=[[0.7, 0.2],  # P(Fever=No | Disease=No), P(Fever=No | Disease=Yes)
                               [0.3, 0.8]], # P(Fever=Yes | Disease=No), P(Fever=Yes | Disease=Yes)
                       evidence=['Disease'], evidence_card=[2])

# Cough depends on Disease
cpd_cough = TabularCPD(variable='Cough', variable_card=2,
                       values=[[0.8, 0.3],  # P(Cough=No | Disease=No), P(Cough=No | Disease=Yes)
                               [0.2, 0.7]], # P(Cough=Yes | Disease=No), P(Cough=Yes | Disease=Yes)
                       evidence=['Disease'], evidence_card=[2])

# Fatigue depends on Disease
cpd_fatigue = TabularCPD(variable='Fatigue', variable_card=2,
                         values=[[0.6, 0.1],  # P(Fatigue=No | Disease=No), P(Fatigue=No | Disease=Yes)
                                 [0.4, 0.9]], # P(Fatigue=Yes | Disease=No), P(Fatigue=Yes | Disease=Yes)
                         evidence=['Disease'], evidence_card=[2])

# Step 3: Adding CPDs to the model
model.add_cpds(cpd_disease, cpd_fever, cpd_cough, cpd_fatigue)

# Step 4: Validate the model structure
assert model.check_model()

# Step 5: Performing inference
inference = VariableElimination(model)

# Query 1: Probability of Disease given evidence that Fever = True and Cough = True
query_result_1 = inference.query(variables=['Disease'], evidence={'Fever': 1, 'Cough': 1})
print("Probability of Disease given Fever=True and Cough=True:\n", query_result_1)

# Query 2: Probability of Disease given evidence that Fatigue = True
query_result_2 = inference.query(variables=['Disease'], evidence={'Fatigue': 1})
print("\nProbability of Disease given Fatigue=True:\n", query_result_2)

# Query 3: Probability of Disease without any evidence
query_result_3 = inference.query(variables=['Disease'])
print("\nProbability of Disease without any evidence:\n", query_result_3)

Probability of Disease given Fever=True and Cough=True:
 +------------+----------------+
| Disease    |   phi(Disease) |
+============+================+
| Disease(0) |         0.1385 |
+------------+----------------+
| Disease(1) |         0.8615 |
+------------+----------------+

Probability of Disease given Fatigue=True:
 +------------+----------------+
| Disease    |   phi(Disease) |
+============+================+
| Disease(0) |         0.4000 |
+------------+----------------+
| Disease(1) |         0.6000 |
+------------+----------------+

Probability of Disease without any evidence:
 +------------+----------------+
| Disease    |   phi(Disease) |
+============+================+
| Disease(0) |         0.6000 |
+------------+----------------+
| Disease(1) |         0.4000 |
+------------+----------------+
